In [1]:
import pickle
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression

import requests
from bs4 import BeautifulSoup
# from sklearn.Logit_model_reloaded import score
# data_test = pd.read_csv("test_notes.csv", sep=',')
# 'Man U','Man C','CHEL','LFC','LEI',"TOT","ARS","EVE","WH","WOL"

In [2]:
team_list_av = []
x_av = []
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}

page = "https://www.transfermarkt.fr/premier-league/startseite/wettbewerb/GB1"
pageTree = requests.get(page, headers=headers)
soup = BeautifulSoup(pageTree.content, 'html.parser')
body = soup.html.find('div',{'class':'large-8 columns'}).find_all('div',{'class':'box'})[1].find('div',{'class':'responsive-table'}).div.table.tbody.find_all('tr')
ixo2 = []
for g in body:

    team_list_av.append(g.find('td',{'hauptlink no-border-links show-for-small show-for-pad'}).a.text)
    ixo = g.find_all('td',{'class':'rechts show-for-small show-for-pad nowrap'})[1].text.replace(' mio. €','').replace(',','.')
    

    if 'mrd' in ixo:
        ixo2.append(float(ixo.replace(' mrd. €',''))*1000)
    else:
        ixo2.append(ixo)
        
    x_av.append(ixo2)

In [3]:
value_model_av = pd.DataFrame({'Team':team_list_av,'Value':x_av[0]})
value_model_av.to_csv('value_model_av_PL.csv',index=False)

In [4]:
value_model_af = pd.read_csv('value_model_av_PL.csv',sep=',')

In [5]:
list_y = []
x = value_model_af['Value']
for i in x:
    print(float(i))
    list_y.append(0.00864271832605479*float(i) + -3.594282721496704)

1040.0
937.25
882.5
879.5
697.0
550.1
548.5
461.75
408.3
391.1
354.75
250.8
248.1
241.4
241.3
239.45
189.55
166.45
145.3
133.0


In [6]:
value_model = pd.DataFrame({'Team':value_model_af['Team'],'Value':value_model_af['Value'],'trend':list_y})

In [7]:
0.00828658469316903*2000 + -3.391241247594151

13.181928138743912

In [8]:
# Récupération du modèle de Régression Logistique 'lr'
with open('model_mhsc.pkl', 'rb') as f:
    my_unpickler = pickle.Unpickler(f)
    model_logit = my_unpickler.load()

    y_pred = model_logit.predict_proba(value_model[['trend']].values) 


In [9]:
for value in y_pred:
    print(value[1])

0.9927554298440714
0.981426630703797
0.96951319888206
0.9686799798662732
0.850570519072479
0.5930673962987459
0.5894812444671749
0.39108814745497483
0.281211132405749
0.2501179893073708
0.19230361944945945
0.08323419657116189
0.08134323584038489
0.07681895444998496
0.07675320762854608
0.07554616394190429
0.048927245669094434
0.039867998353660436
0.03300123477758126
0.029548500125705115


In [10]:
# predictions = [round(value) for value in y_pred]
# print(ytest)
# precision = model_logit.score(value_model[['Value','trend']],predictions)
# print(np.around(precision,decimals=2)*100,'%')

In [11]:
y_pred

array([[0.00724457, 0.99275543],
       [0.01857337, 0.98142663],
       [0.0304868 , 0.9695132 ],
       [0.03132002, 0.96867998],
       [0.14942948, 0.85057052],
       [0.4069326 , 0.5930674 ],
       [0.41051876, 0.58948124],
       [0.60891185, 0.39108815],
       [0.71878887, 0.28121113],
       [0.74988201, 0.25011799],
       [0.80769638, 0.19230362],
       [0.9167658 , 0.0832342 ],
       [0.91865676, 0.08134324],
       [0.92318105, 0.07681895],
       [0.92324679, 0.07675321],
       [0.92445384, 0.07554616],
       [0.95107275, 0.04892725],
       [0.960132  , 0.039868  ],
       [0.96699877, 0.03300123],
       [0.9704515 , 0.0295485 ]])

In [12]:
predict = []
prt_predict = []
for i in y_pred:
    prt_predict.append(i[1])
value_model['pourcentage'] = prt_predict

proba = 6
count = 0
proba_list = []
high_count = 0
for a,k,i in zip(value_model.sort_values(by='pourcentage',ascending=False)['trend'],value_model.sort_values(by='pourcentage',ascending=False)['Team'],value_model.sort_values(by='pourcentage',ascending=False)['pourcentage']):
    if (proba == 6)&(a < 4):
        proba = proba - high_count
    
    proba_list.append(np.around(i*(proba/6),decimals=3))
    print(proba,count,high_count)
    count = count + 1
    if a > 4:
        high_count = high_count + 1
        continue
    
    if proba < 3:
        if count == 10:
            proba = proba - 1
        else:
            continue
    else:
        proba = proba - 1
        
value_model['pourcentage'] = proba_list
        
for i in list(value_model['pourcentage']):
    if i < 0.5:
        predict.append(False)
    elif i > 0.5:
        predict.append(True)
    
#print(len(predict))
#print(len(y_pred[1]))
value_model['predict'] = predict


6 0 0
6 1 1
6 2 2
6 3 3
2 4 4
2 5 4
2 6 4
2 7 4
2 8 4
2 9 4
1 10 4
1 11 4
1 12 4
1 13 4
1 14 4
1 15 4
1 16 4
1 17 4
1 18 4
1 19 4


In [13]:
print('Prédictions :', value_model['Team'][0],' : ', np.around(y_pred[0][1],decimals=2),' %')
print('Prédictions :', value_model['Team'][1],' : ', np.around(y_pred[1][1],decimals=2),' %')
print('Prédictions :', value_model['Team'][2],' : ', np.around(y_pred[2][1],decimals=2),' %')
print('Prédictions :', value_model['Team'][3],' : ', np.around(y_pred[3][1],decimals=2),' %')
print('Prédictions :', value_model['Team'][4],' : ', np.around(y_pred[4][1],decimals=2),' %')

Prédictions : Manchester City  :  0.99  %
Prédictions : Manchester Utd.  :  0.98  %
Prédictions : Chelsea  :  0.97  %
Prédictions : FC Liverpool  :  0.97  %
Prédictions : Tottenham  :  0.85  %


In [14]:
# value_model = pd.read_csv('model_result_PL.csv',sep=',')

In [15]:
# proba = 6
# for k,i in zip(value_model.sort_values(by='pourcentage',ascending=False)['Team'],value_model.sort_values(by='pourcentage',ascending=False)['pourcentage']):
#     print(k,np.around(i*(proba/6),decimals=3))
#     if proba < 3:
#         continue
#     else:
#         proba = proba - 1

In [16]:
value_model

,Team,Value,trend,pourcentage,predict
0,Manchester City,1040.00,5.394144,0.993,True
1,Manchester Utd.,937.25,4.506105,0.981,True
2,Chelsea,882.50,4.032916,0.970,True
3,FC Liverpool,879.50,4.006988,0.969,True
4,Tottenham,697.00,2.429692,0.284,False
5,Leicester City,550.10,1.160077,0.198,False
6,FC Arsenal,548.50,1.146248,0.196,False
7,FC Everton,461.75,0.396492,0.130,False
8,Aston Villa,408.30,-0.065461,0.094,False
9,Wolverhampton,391.10,-0.214116,0.083,False


In [17]:
value_model.to_csv('../model_result_PL.csv',index=False)